Some details found here:

http://elasticsearch-py.readthedocs.io/en/master/

In [2]:

from pyelasticsearch.client import ElasticSearch

In [3]:
endpoint = 'https://search-test-vu2fwve5ykzm5tsjkut5q4idum.us-east-1.es.amazonaws.com'

In [4]:
es = ElasticSearch(endpoint, port=443)

In [5]:
es.index('contacts',
          'person',
          {'name': 'Jing', 'age': 25, 'title': 'QA Master'},
           id=6)

{'_id': '6',
 '_index': 'contacts',
 '_shards': {'failed': 0, 'successful': 1, 'total': 2},
 '_type': 'person',
 '_version': 1,
 'created': True,
 'result': 'created'}

In [5]:
docs = [{'id': 2, 'name': 'Jessica Coder', 'age': 32, 'title': 'Programmer'},
          {'id': 3, 'name': 'Freddy Tester', 'age': 29, 'title': 'Office Assistant'}]
es.bulk((es.index_op(doc, id=doc.pop('id')) for doc in docs),
          index='contacts',
          doc_type='person')

{'errors': False,
 'items': [{'index': {'_id': '2',
    '_index': 'contacts',
    '_shards': {'failed': 0, 'successful': 1, 'total': 2},
    '_type': 'person',
    '_version': 7,
    'created': False,
    'result': 'updated',
    'status': 200}},
  {'index': {'_id': '3',
    '_index': 'contacts',
    '_shards': {'failed': 0, 'successful': 1, 'total': 2},
    '_type': 'person',
    '_version': 5,
    'created': False,
    'result': 'updated',
    'status': 200}}],
 'took': 17}

In [16]:
es.refresh('contacts')

{'_shards': {'failed': 0, 'successful': 5, 'total': 10}}

In [17]:
es.get('contacts', 'person', 3)

{'_id': '3',
 '_index': 'contacts',
 '_source': {'age': 29, 'name': 'Freddy Tester', 'title': 'Office Assistant'},
 '_type': 'person',
 '_version': 3,
 'found': True}

In [19]:
es.search('name:joe OR name:freddy', index='contacts')

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 25, 'name': 'Joe Tester', 'title': 'QA Master'},
    '_type': 'person'},
   {'_id': '3',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 29,
     'name': 'Freddy Tester',
     'title': 'Office Assistant'},
    '_type': 'person'}],
  'max_score': 0.25811607,
  'total': 2},
 'timed_out': False,
 'took': 2}

In [39]:
es.search(index='contacts', query={"query": {"match": {"name": "Tester"}}})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '1',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 25, 'name': 'Joe Tester', 'title': 'QA Master'},
    '_type': 'person'},
   {'_id': '3',
    '_index': 'contacts',
    '_score': 0.25811607,
    '_source': {'age': 29,
     'name': 'Freddy Tester',
     'title': 'Office Assistant'},
    '_type': 'person'}],
  'max_score': 0.25811607,
  'total': 2},
 'timed_out': False,
 'took': 4}

In [25]:
query = {
     'query': {
         'filtered': {
             'query': {
                 'query_string': {'query': 'name:tester'}
             },
             'filter': {
                 'range': {
                     'age': {
                         'from': 27,
                         'to': 37,
                     },
                 },
             },
         },
     },
}
es.search(index='contacts', query=query)

GET /contacts/_search [status:400 request:1.011s]


ElasticHttpError: (400, 'parsing_exception')

In [27]:
es.delete_index('contacts')

{u'acknowledged': True}

In [ ]:
import elasticsearch

host = 'YOURHOST.us-east-1.es.amazonaws.com'
awsauth = AWS4Auth(your_access_key, your_secret_key, region, 'es')

es = elasticsearch.Elasticsearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=elasticsearch.connection.RequestsHttpConnection
)
print(es.cluster.health())

In [34]:
from elasticsearch import Elasticsearch
import json
import requests
from pyelasticsearch.client import ElasticSearch

search_v1={ 
    'mapping':{
        'blog':{
            'properties':{
                'blog_id':{'type':'integer'},
                'title':{'type':'string','analyzer':'english'},
                'author':{'type':'string','index':'not_analyzed'},
                'abstract':{'type':'string','analyzer':'english'},
                'date_created':{'type':'date'},
                'publish_date':{'type':'date'},
                'content':{'type':'string','analyzer':'english'}
            }
        }
    }
}



if __name__ == '__main__':
    config = json.load(open('../config/configsql.json', 'r'))
    # TODO: Jing to populate some code from the notebook
    user = config['dev']['mysql']['user']
    password = config['dev']['mysql']['password']
    host = config['dev']['mysql']['host']
    port = config['dev']['mysql']['port']
    dbname = config['dev']['mysql']['dbname']
    aws_access_key_id=config['dev']['aws']['key']
    aws_secret_access_key=config['dev']['aws']['secret']
    bucketname=config['dev']['bucketname']['bucketname']
    endpoint = 'https://search-test-vu2fwve5ykzm5tsjkut5q4idum.us-east-1.es.amazonaws.com'
    client = Elasticsearch(endpoint, port=443)
    client.indices.create(index='search_v1')
   

/usr/local/lib/python3.6/site-packages/elasticsearch/connection/http_urllib3.py:54: UserWarning: Connecting to search-test-vu2fwve5ykzm5tsjkut5q4idum.us-east-1.es.amazonaws.com using SSL with verify_certs=False is insecure.
  'Connecting to %s using SSL with verify_certs=False is insecure.' % host)
/usr/local/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [31]:
client.indices.delete(index='search_v1')

/usr/local/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'acknowledged': True}

In [ ]:
successful_update = False
#result = elastic_search_conn.delete(index=index_name,doc_type='blog',body='{"query":{"match_all":{}}}')
doc = {
    'blog_id'      : int(row['blog_id']),
    'title'        : row['title'],
    'author'       : row['author'],
    'abstract'     : row['abstract'],
    'date_created' : row['date_created'],
    'publish_date' : row['publish_date'],
    'content'      : html
    }
print("Going to update " + row['title'])
result2 = elastic_search_conn.create(index='search_v1',doc_type='blog',body=doc) 

In [6]:
es.get('search_v1', 'blog', 3)

{'_id': '3',
 '_index': 'search_v1',
 '_source': {'abstract': 'In this episode we discuss the techniques and approaches to perform advertising attribution - the process of helping advertisers understand the impact all aspects of their marketing efforts contribute to their overall g',
  'author': 'Kyle',
  'blog_id': 3,
  'content': '<h2>Advertising Attribution with Nathan Janos</h2>\n<p>In this episode we discuss the techniques and approaches to perform advertising attribution - the process of helping advertisers understand the impact all aspects of their marketing efforts contribute to their overall gain.</p>\n<h3>Links to things mentioned during the show:</h3>\n<p><a href="http://longnow.org/">The Long Now Foundation</a></p>\n<p><a href="http://jeffjag.com/">Jeff Jagunich, artist</a></p>\n<p><a href="http://www.convertro.com/">Convertro</a></p>',
  'date_created': '2014-06-06T00:00:00',
  'prettyname': '/episodes/2014/advertising-attribution-with-nathan-janos',
  'publish_date': '201

In [7]:
es.get('search_v1', 'blog', 1)

{'_id': '1',
 '_index': 'search_v1',
 '_source': {'abstract': 'The Data Skeptic Podcast features conversations with topics related to data science, statistics, machine learning, artificial intelligence and the like, all from the perspective of applying critical thinking and the scientific method to evaluate the veracity of claims and efficacy of approac',
  'author': 'Kyle',
  'blog_id': 1,
  'content': '<h2>Introduction</h2>\n<p>The Data Skeptic Podcast features conversations with topics related to data science, statistics, machine learning, artificial intelligence and the like, all from the perspective of applying critical thinking and the scientific method to evaluate the veracity of claims and efficacy of approaches.</p>',
  'date_created': '2014-05-23T00:00:00',
  'prettyname': '/episodes/2014/introduction',
  'publish_date': '2014-05-23T00:00:00',
  'title': 'Introduction'},
 '_type': 'blog',
 '_version': 20,
 'found': True}

In [10]:
es.search(index='search_v1',
					query={
					  "query": {
					    "multi_match": {
					      "query": 'beer',
					      "fields": [
					        "title",
					        "abstract"
					      ]
					    }
					  },
					  "_source": {
					    "includes": [
					      "blog_id",
					      "title",
					      "abstract",
					      "prettyname",
					      "publish_date"
					    ]
					  }
					})

{'_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '336',
    '_index': 'search_v1',
    '_score': 3.598687,
    '_source': {'abstract': 'For some interactive clustering, check out the presentation given at the Oktoberfest 2017 RLadies Chicago meetup or the Shiny app built from the same data',
     'blog_id': 336,
     'prettyname': '/projects/2018/beer-in-hand-data-science',
     'publish_date': '2099-01-01T00:00:00',
     'title': 'Beer-in-Hand Data Science'},
    '_type': 'blog'},
   {'_id': '335',
    '_index': 'search_v1',
    '_score': 2.6939664,
    '_source': {'abstract': 'For some interactive clustering, check out the presentation given at the Oktoberfest 2017 RLadies Chicago meetup or the Shiny app built from the same data',
     'blog_id': 335,
     'prettyname': '/projects/2018/beer-in-hand-data-science',
     'publish_date': '2018-02-04T00:00:00',
     'title': 'Beer-in-Hand Data Science'},
    '_type': 'blog'},
   {'_id': '338',
    '_index